## Her skal det være en relevant overskrift.

#### Innhenter verktøy fra bibliotek
Import-stegene henter inn bibliotek med kode og funksjoner utviklet ekstern.

In [ ]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

#### Kjører metoden read.path for å få oversikt over hvilke parquet datasett som er tilrettelagt i tilknytning veilderens lagringsområde i sky.  Oversikt blir lest inn i egen dataframe - df_datsets.
Aktuelt lagringsområde blir lagt inn som parameter (string objekt som vi definerer selv) INNDATA_PATH.

In [ ]:
INNDATA_PATH = '/user/matz.ivan.faldmo/*'
df_datasets = spark.read.path(INNDATA_PATH)


df_datsets skrives ut i output vindu. 

In [ ]:
df_datasets.show(100, False)

#### Leser inn parquet datasett
Leser inn parquet datsett df_sammensatt_pyspark, selekterer de variable vi skal bruke og etablerer dataframen df

In [ ]:
df = spark.read.path('/user/matz.ivan.faldmo/df_sammensatt_pyspark').select('Land', 'Areal', 'BNP', 'Innbyggerantall')
df.show()

#### Kjører ut statistiske størrelser
Kjører ut statistike størrelser fra datframe df med metoden describe. Resultat fr metode blir lagt i dataframeen df_stat.

In [ ]:
df_stat = df.describe()
df_stat.show()

Transformerer df_stat (med bl.a. pivot funksjonen) slik at rader i df_stat blir kolonner i dataframe df_stat_piv. Formål er å gjøre output mer oversiktlig ved at alle størrelser på en gitt variabel blir liggende i samme rad. Dette kan være hensiktsmessig i de tilfeller antall "statistikkvaiable" er høyt (dvs langt høyere enn de fire som inngår i eksempel.

In [ ]:
var_lst = list(df_stat.columns)
var_lst.remove('summary')
i = 1
for col in var_lst:
    df = df_stat.groupBy().pivot('summary').agg(F.sum(col)).withColumn('variabel',F.lit(col))
    if i == 1:
        df_stat_piv = df 
    else:
        df_stat_piv = df_stat_piv.unionByName(df)
    i = i + 1

In [ ]:
df_stat_piv.show()

Kjører ut median (ikke fullført)

In [ ]:
liste_kolonner_median = ['Areal', 'Innbyggerantall', 'BNP']
df_median = df.approxQuantile(liste_kolonner_median, [0.5], 0)
print(df_median)